In [ ]:
# from google.colab import drive
# drive.mount('/content/drive',force_remount=True)


In [ ]:
# %cd drive/My\ Drive/SIC\ -\ BigData/8.\ Project/

In [ ]:
# %ls

## Import packages

In [1]:
# !pip install evaluate
# !pip install -U accelerate
!pip install datasets
# !pip install lxml
# !pip install -U pandas
# !pip install numpy
# !pip install transformers
# !pip install torch
# !pip install --upgrade pyarrow
# !pip install scikit-learn
# !pip install tensorboardX
# !pip install seaborn
# !pip install spacy
# !pip install xgboost
# !pip install beautifulsoup4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
import numpy as np
import pandas as pd
import torch
from datasets import Dataset, DatasetDict
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, AdamW, AutoModel, AutoModelForSequenceClassification, get_linear_schedule_with_warmup, AutoTokenizer
from transformers import AutoTokenizer
from datasets import load_dataset
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch import nn
import spacy
import re
import string
import string
from bs4 import BeautifulSoup
from spacy.lang.en.stop_words import STOP_WORDS
stop_words = STOP_WORDS
punctuations = string.punctuation

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

## Prepare DataSet

In [4]:
model_name = "bert-base-cased"
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)


ds = load_dataset('takala/financial_phrasebank', 'sentences_50agree',trust_remote_code=True)
df = ds['train'].to_pandas()
# df = df.drop_duplicates(subset='sentence')
df.rename(columns={'sentence':'text'},inplace=True)

# 0 = negative
# 1 = neutral
# 2 = positive

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4846 [00:00<?, ? examples/s]

In [5]:
df

,text,label
0,"According to Gran , the company has no plans t...",1
1,Technopolis plans to develop in stages an area...,1
2,The international electronic industry company ...,0
3,With the new production plant the company woul...,2
4,According to the company 's updated strategy f...,2
...,...,...
4841,LONDON MarketWatch -- Share prices ended lower...,0
4842,Rinkuskiai 's beer sales fell by 6.5 per cent ...,1
4843,Operating profit fell to EUR 35.4 mn from EUR ...,0
4844,Net sales of the Paper segment decreased to EU...,0


In [6]:
# Load dataset
temp_df, test_df = train_test_split(df, test_size=0.20, stratify=df['label'], random_state=42)
train_df, val_df = train_test_split(temp_df, test_size=0.20, stratify=temp_df['label'], random_state=42)

In [7]:
# Chuyển đổi ngược lại từ DataFrame sang Dataset
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
val_dataset = Dataset.from_pandas(val_df.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True))

In [8]:
dataset = DatasetDict()
dataset['train'] = train_dataset
dataset['val'] = val_dataset
dataset ['test'] = test_dataset

## Preprocessing DataSet

#### A: Clean 'text' columns
- python -m spacy download en_core_web_sm

**Creating our tokenizer function - lemmatized, lowercased; remove "-PRON"s, stop_words, punctuations**

In [9]:

# nlp = English()
nlp = spacy.load('en_core_web_sm')
def clean_text(sentence):
    # # Parse HTML and extract plain text
    soup = BeautifulSoup(sentence+'<html>', "html.parser")
    sentence = re.sub(r'\[[^]]*\]', '', soup.get_text())

    # # # Remove links
    sentence = re.sub(r'http\S+|www\S+|https\S+', '', sentence, flags=re.MULTILINE)

    # # Remove words with invalid characters (keeping alphanumeric and %,'.')
    # sentence = ' '.join(word for word in sentence.split() if re.match(r'^[a-zA-Z0-9%.,:]+$', word))

    # # Process text with spaCy
    # doc = nlp(sentence)
    # mytokens = [word.lemma_.strip() if word.lemma_ != "-PRON-" else word for word in doc]

    # # Remove stop words and punctuation
    # mytokens = [word.text for word in doc if word.text.lower() not in stop_words and word.text not in punctuations]

    # Return cleaned text
    # return ' '.join(mytokens).strip()
    return sentence

In [10]:
print(clean_text('$QCOM chart: Qualcomm Short 61.8% Retracement, Trendline and Resistance. https://t.co/2WQqh35KOY'))

$QCOM chart: Qualcomm Short 61.8% Retracement, Trendline and Resistance. 


**Clean_text**

In [11]:
def preprocess_function(examples):
    examples['text'] = [clean_text(text) for text in examples['text']]
    return examples
cleaned_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/3100 [00:00<?, ? examples/s]

Map:   0%|          | 0/776 [00:00<?, ? examples/s]

Map:   0%|          | 0/970 [00:00<?, ? examples/s]

In [12]:
# Preprocess function
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=64)

encoded_dataset = cleaned_dataset.map(tokenize, batched=True,batch_size=None)

Map:   0%|          | 0/3100 [00:00<?, ? examples/s]

Map:   0%|          | 0/776 [00:00<?, ? examples/s]

Map:   0%|          | 0/970 [00:00<?, ? examples/s]

In [13]:
encoded_dataset = encoded_dataset.remove_columns('text')

In [14]:
encoded_dataset['test']

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 970
})

## Model

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


In [16]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3).to(device)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
class CustomClassifier(nn.Module):
    def __init__(self):
        super(CustomClassifier, self).__init__()

        self.fc1 = nn.Linear(768, 256)
        self.bn = nn.BatchNorm1d(256)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(256, 3)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.softmax(x)

        return x

model.classifier = CustomClassifier()

**Define Trainer parameters**

In [18]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    eval_strategy='epoch',
    save_strategy="epoch",
    disable_tqdm = False,

    greater_is_better=False,
    load_best_model_at_end = True,
    metric_for_best_model='eval_loss',

    # fp16=True,
    # overwrite_output_dir=True,
    # seed=42,
    # local_rank=-1,
    # gradient_accumulation_steps=1,
    # warmup_steps=500,
)

# Compute metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['val'],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1, early_stopping_threshold=.0)],
    optimizers= (torch.optim.AdamW(model.parameters(), lr=2e-5, eps=1e-5, betas=(0.9, 0.999)), None),
)

In [ ]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
trainer.evaluate()

In [ ]:
trainer.predict(encoded_dataset['test'])